# INTRODUCTION

## Contexte du Projet

## Mise en place de PySpark

In [1]:
# Installation de pyspark
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


IMPORTANT : PENSEZ A INSTALLER JAVA - winget install --exact --id EclipseAdoptium.Temurin.11.JDK

Fix pour python 3.12 qui ne contient plus la librairie distutils

In [2]:
import sys
try:
    import distutils
except ModuleNotFoundError:
    import setuptools._distutils as distutils
    sys.modules["distutils"] = distutils

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MoviesRatings") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


25/03/07 10:27:44 WARN Utils: Your hostname, geley-GS65-Stealth-8SF resolves to a loopback address: 127.0.1.1; using 10.84.120.70 instead (on interface wlo1)
25/03/07 10:27:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 10:27:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Importation des librairies


In [4]:
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display
from pyspark.sql.functions import isnan, when, count, col, avg, lit, explode, split, row_number
from pyspark.sql.window import Window
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import collect_list

/home/geley/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Import et traitement des datasets

In [5]:
# Import des datasets ratings.csv & movies.csv
ratings = spark.read.csv("ml-latest-small/ratings.csv", header=True, inferSchema=True)
movies = spark.read.csv("ml-latest-small/movies.csv", header=True, inferSchema=True)


In [6]:
# On Affiche les les 10 premières lignes des deux datasets
ratings.show(10)
movies.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|

In [7]:
# On affiche le schéma des deux datasets
ratings.printSchema()
movies.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [8]:
# On regarde le nombre de lignes de chaque dataset
print("Nombre de lignes dans ratings : ", ratings.count())
print("Nombre de lignes dans movies : ", movies.count())

Nombre de lignes dans ratings :  100836
Nombre de lignes dans movies :  9742


In [9]:
# On regarde le nombre de valeurs manquantes dans chaque dataset

ratings.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ratings.columns]).show()
movies.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in movies.columns]).show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     0|      0|     0|        0|
+------+-------+------+---------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|      0|    0|     0|
+-------+-----+------+



In [10]:
# On regarde le nombre de valeurs distinctes dans chaque dataset
ratings.agg(*(count(col(c)).alias(c) for c in ratings.columns)).show()
movies.agg(*(count(col(c)).alias(c) for c in movies.columns)).show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|100836| 100836|100836|   100836|
+------+-------+------+---------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|   9742| 9742|  9742|
+-------+-----+------+



In [11]:
# On regarde les statistiques descriptives des colonnes rating et timestamp du dataset ratings
ratings.describe(["rating", "timestamp"]).show()


+-------+------------------+--------------------+
|summary|            rating|           timestamp|
+-------+------------------+--------------------+
|  count|            100836|              100836|
|   mean| 3.501556983616962|1.2059460873684695E9|
| stddev|1.0425292390606342|2.1626103599513078E8|
|    min|               0.5|           828124615|
|    max|               5.0|          1537799250|
+-------+------------------+--------------------+



In [12]:
# Quels sont les films les mieux notés en moyenne 

ratings.groupBy("movieId").agg(avg("rating").alias("avg_rating")).orderBy("avg_rating", ascending=False).show()

+-------+----------+
|movieId|avg_rating|
+-------+----------+
|  26350|       5.0|
|   3795|       5.0|
|  25887|       5.0|
| 157775|       5.0|
|    633|       5.0|
|  33138|       5.0|
|  67618|       5.0|
|    876|       5.0|
|    496|       5.0|
|  27373|       5.0|
| 113829|       5.0|
|  53578|       5.0|
| 152711|       5.0|
| 118894|       5.0|
|     53|       5.0|
| 160644|       5.0|
|    148|       5.0|
|   8911|       5.0|
| 147300|       5.0|
|  84273|       5.0|
+-------+----------+
only showing top 20 rows



In [13]:
# Quels sont les films les plus populaires 
ratings.groupBy("movieId").count().orderBy("count", ascending=False).show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [14]:
# Quels sont les 10 films les mieux notés en moyenne avec plus de 100/200 & 300 notes

# Jointure commune entre ratings et movies
ratings_with_titles = ratings.join(movies, on="movieId", how="inner")

# Fonction pour obtenir les films mieux notés avec un seuil minimal de votes
def top_movies_by_avg(threshold, limit=10):
    return (
        ratings_with_titles
        .groupBy("movieId", "title")
        .agg(avg("rating").alias("avg_rating"), count("rating").alias("count_rating"))
        .filter(col("count_rating") > threshold)
        .orderBy(col("avg_rating").desc())
        .select("title", "avg_rating", "count_rating")
        .limit(limit)
    )

top_movies_by_avg(100).show(10, False)
top_movies_by_avg(200).show(10, False)
top_movies_by_avg(300).show(10, False)

+-----------------------------------------+-----------------+------------+
|title                                    |avg_rating       |count_rating|
+-----------------------------------------+-----------------+------------+
|Shawshank Redemption, The (1994)         |4.429022082018927|317         |
|Godfather, The (1972)                    |4.2890625        |192         |
|Fight Club (1999)                        |4.272935779816514|218         |
|Godfather: Part II, The (1974)           |4.25968992248062 |129         |
|Departed, The (2006)                     |4.252336448598131|107         |
|Goodfellas (1990)                        |4.25             |126         |
|Dark Knight, The (2008)                  |4.238255033557047|149         |
|Usual Suspects, The (1995)               |4.237745098039215|204         |
|Princess Bride, The (1987)               |4.232394366197183|142         |
|Star Wars: Episode IV - A New Hope (1977)|4.231075697211155|251         |
+------------------------

In [15]:
# Quels genres de films sont les plus populaires ? (nb de notes) & Top 5 des films les plus populaires par genre

def most_popular_movies(limit=30):
    return (
        ratings.groupBy("movieId")
               .agg(count("rating").alias("count_rating"))
               .join(movies, "movieId")
               .orderBy(col("count_rating").desc())
               .limit(limit)
    )

def top5_by_genre():
    movies_with_genre = movies.withColumn("genre", explode(split(col("genres"), "\\|")))
    movies_ratings = ratings.join(movies_with_genre, "movieId")
    movies_stats = movies_ratings.groupBy("title", "genre") \
        .agg(count("rating").alias("count_rating"), avg("rating").alias("avg_rating"))
    windowSpec = Window.partitionBy("genre").orderBy(col("count_rating").desc())
    return movies_stats.withColumn("rank", row_number().over(windowSpec)) \
        .filter(col("rank") <= 5) \
        .orderBy("genre", "rank")

def display_top5_by_genre(limit_per_genre=5):
    df = top5_by_genre()
    # Récupérer la liste des genres distincts
    genres = [row["genre"] for row in df.select("genre").distinct().collect()]
    for genre in sorted(genres):
        print(f"Genre: {genre}")
        df.filter(col("genre") == genre).orderBy("rank").show(limit_per_genre, False)

# Utilisation :

print("Most Popular Movies:")
most_popular_movies().show(30, False)

print("\nTop 5 Movies by Genre:")
display_top5_by_genre()

Most Popular Movies:
+-------+------------+------------------------------------------------------------------------------+-------------------------------------------+
|movieId|count_rating|title                                                                         |genres                                     |
+-------+------------+------------------------------------------------------------------------------+-------------------------------------------+
|356    |329         |Forrest Gump (1994)                                                           |Comedy|Drama|Romance|War                   |
|318    |317         |Shawshank Redemption, The (1994)                                              |Crime|Drama                                |
|296    |307         |Pulp Fiction (1994)                                                           |Comedy|Crime|Drama|Thriller                |
|593    |279         |Silence of the Lambs, The (1991)                                              |Cr

## Modelisation avec Spark MLlib : ALS

In [16]:
# S'assurer que les colonnes ont le bon type
ratings = ratings.withColumn("userId", col("userId").cast("integer")) \
                 .withColumn("movieId", col("movieId").cast("integer")) \
                 .withColumn("rating", col("rating").cast("float"))

# Diviser en 80% training et 20% test
(training, test) = ratings.randomSplit([0.8, 0.2])

In [17]:
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=15,           # Nombre d'itérations
    regParam=0.095,         # Paramètre de régularisation
    rank=10,              # Dimension des facteurs latents
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating",
    coldStartStrategy="drop"  # Pour ignorer les prédictions NaN lors de l'évaluation
)

# Entraîner le modèle sur le jeu de données d'entraînement
model = als.fit(training)

25/03/07 10:28:08 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/03/07 10:28:08 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [18]:
from pyspark.ml.evaluation import RegressionEvaluator

# Générer des prédictions sur le test set
predictions = model.transform(test)

# Configurer l'évaluateur RMSE
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating", 
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = {}".format(rmse))


Root-mean-square error = 0.8862959241619184


In [19]:
# Générer des recommandations pour tous les utilisateurs
userRecs = model.recommendForAllUsers(10)
userRecs.show(10, False)

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                               |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |[{33649, 5.9002757}, {5075, 5.6979656}, {78836, 5.656192}, {3653, 5.65577}, {1209, 5.643887}, {3030, 5.6412706}, {184245, 5.60833}, {179135, 5.60833}, {171495, 5.60833}, {138966, 5.60833}]  |
|2     |[{918, 5.2671285}, {3200, 5.2242723}, {33649, 5.218885}, {905, 5.021516}, {7767, 4.949567}, {3022, 4.930159}, {6201, 4.9170294}, {4495, 4.9170294}, {131724, 4.911894}, {3451, 4.8825135}]    |


In [20]:
# Générer les recommandations pour un utilisateur spécifique
userRecs = model.recommendForUserSubset(ratings.filter(col("userId") == 123), 5)
userRecs = userRecs.withColumn("rec", explode(col("recommendations"))) \
                   .select(col("userId"), col("rec.movieId"), col("rec.rating")) \
                   .join(movies, "movieId", "left") \
                   .select("userId", "title", "rating")

userRecs.show(truncate=False)

+------+-----------------------+---------+
|userId|title                  |rating   |
+------+-----------------------+---------+
|123   |Saving Face (2004)     |4.992414 |
|123   |Waydowntown (2000)     |4.836974 |
|123   |De platte jungle (1978)|4.8061843|
|123   |Blue Planet II (2017)  |4.8061843|
|123   |Cosmos                 |4.8061843|
+------+-----------------------+---------+



## Recommandation basée sur le contenu

In [25]:
# Charger les données
movies = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")

# Exploration des données
movies.head(), ratings.head()

# Traitement des genres
def clean_genres(genres):
    return " ".join(genres.split('|')) if isinstance(genres, str) else ""

movies["clean_genres"] = movies["genres"].apply(clean_genres)

# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(movies["clean_genres"])

# Calcul de la similarité cosinus
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Création du mapping des titres vers les index
indices = pd.Series(movies.index, index=movies["title"]).drop_duplicates()

def recommend_movies(title, n=5):
    if title not in indices:
        return ["Film non trouvé. Essayez un autre titre."]
    
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    movie_indices = [i[0] for i in sim_scores]
    return movies.iloc[movie_indices]["title"].tolist()

# Interface interactive
output = widgets.Output()
title_dropdown = widgets.Combobox(
    options=movies["title"].tolist(),
    description='Film:',
    ensure_option=True
)

def on_change(change):
    with output:
        output.clear_output(wait=True)
        if change["type"] == "change" and change["name"] == "value":
            recommendations = recommend_movies(change["new"])
            print(f"\nFilms similaires à '{change['new']}':")
            for rec in recommendations:
                print(f"- {rec}")

title_dropdown.observe(on_change, names='value')
display(title_dropdown, output)

Combobox(value='', description='Film:', ensure_option=True, options=('Toy Story (1995)', 'Jumanji (1995)', 'Gr…

Output()

## Recommandation basée sur les proximités utilisateurs (KNN)

In [22]:
# Initialisation de Spark
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# Charger les données
df = spark.read.csv("ml-latest-small/movies.csv", header=True, inferSchema=True)

# Transformer les genres en liste de genres
def split_genres(genres_str):
    return genres_str.split('|') if genres_str else []

# Enregistrer la fonction en tant que UDF pour PySpark
split_genres_udf = udf(split_genres, ArrayType(StringType()))
df = df.withColumn("genres", split_genres_udf(df["genres"]))

# Convertir le DataFrame PySpark en DataFrame Pandas
df_pandas = df.select("title", "genres").toPandas()

# Utilisation de MultiLabelBinarizer de sklearn pour encoder les genres
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df_pandas["genres"])
X = pd.DataFrame(genres_encoded, columns=mlb.classes_)

# Initialisation du modèle KNN de sklearn
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(X)

# Fonction de recommandation
def recommend_movies(movie_title):
    # Trouver l'index du film par titre
    idx = df_pandas[df_pandas["title"] == movie_title].index[0]
    # Obtenir les indices des films les plus similaires
    distances, indices = knn.kneighbors([X.iloc[idx]])
    # Extraire les titres des films recommandés
    recommendations = df_pandas.iloc[indices[0][1:]]["title"].values
    return recommendations

# Exemple de recommandation
movie_example = "Toy Story (1995)"
recommended_movies = recommend_movies(movie_example)
print(f"Films similaires à '{movie_example}': {recommended_movies}")

25/03/07 10:28:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Films similaires à 'Toy Story (1995)': ['Monsters, Inc. (2001)' 'Moana (2016)'
 'Asterix and the Vikings (Astérix et les Vikings) (2006)' 'Turbo (2013)']


/home/geley/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


## Evaluation des approches de recommandation